# 第一題[myknn_regressor]

In [135]:
# given code
import pickle
from sklearn import preprocessing
import numpy as np

# Load data
with open('msd_data1.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test']

### Q1.1 myknn_regressor 的程式碼

依據題目說明，使用 BruteForce 計算 X_test 中每首歌曲與 X_train 中所有歌曲的特徵距離。
每個 X_test 掃過所有的 X_train 之後，會得到 Y_train 中距離最近的 k 個年份。
依據輸入的 mean_type 分別使用 equal_weight 或 remove_outliers 來計算 Y_test 的結果。
最後會得到我們計算的 Y_test 並回傳 ypred 完成此部分

In [144]:
class myknn_regressor():
    def __init__(self, n_neighbors, mean_type):
        """mean_type can be equal_weight or remove_outliers.
                              equal_weight use the same weight for all neighbors.
                              remove_outliers remove neighbors out in [Q1 - 1.5 IQR, Q3 + 1.5IQR]."""
        self.n_neighbors = n_neighbors
        self.mean_type = mean_type

    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x_test):
        """use remove_outliers only if k>=10"""
        self.x_test = x_test
        k = self.n_neighbors
        x_train = self.x_train
        y_train = self.y_train
        mean_type = self.mean_type
        max_dis = 5.0 #測試與記錄用
        y_predict = np.empty((len(x_test),1), dtype=int) #建立一個存結果的 y_predict
        
        for a in range(len(x_test)):

            neighbors = np.empty((k,2),dtype=float )
            test_counter = 0
            dis_temp = 0.0
            
            for j in x_train:
                
                # 第一部分先把 neighbors 填滿
                
                if(test_counter<k): #前k個 x_train
                    for l in range(len(j)):
                        dis_temp = dis_temp + (((x_test[a][l]-j[l])**2)**0.5) #計算兩個點之間的距離
                    dis_temp = dis_temp/len(j) #平均
                    if (dis_temp<max_dis): #更新極值
                        max_dis = dis_temp
                    neighbors[test_counter][0]=dis_temp
                    neighbors[test_counter][1]=y_train[test_counter]
                
                # 第二部分跟 neighbors 比較距離，如果距離比 neighbors 中的更短就更新
                
                else:
                    for l in range(len(j)):
                        dis_temp = dis_temp + (((x_test[a][l]-j[l])**2)**0.5) #計算兩個點之間的距離
                    dis_temp = dis_temp/len(j) #平均
                    # 開始跟前k個比較，有沒有比前k個中最遠的更近，有的話就更新
                    if (dis_temp<max_dis): #更新極值 note:有必要這個步驟嗎？
                        max_dis = dis_temp
                    for i in range(k):
                        if(dis_temp < neighbors[i][0]):
                            neighbors[i][0]=dis_temp
                            neighbors[i][1]=y_train[test_counter]
                            break
                
                # 更新參數
                test_counter+=1
                dis_temp = 0.0

            # 判斷 mean_type
            if(mean_type == "remove_outliers"):

                #新增一個 array 排序，找出 quantile
                quantile = np.empty((k,1),dtype=float)
                ro_outcome = 0
                for i in range(k):
                    ro_outcome += neighbors[i][1]
                    quantile[i] = neighbors[i][1]

                # bubble sort
                for m in range(k):
                    for n in range(k-1):
                        if(quantile[n]>quantile[n+1]):
                            temp = quantile[n]
                            quantile[n] = quantile[n+1]
                            quantile[n+1] = temp

                # 計算出 outlier 的範圍
                Q1 = int((k+1)*0.25)
                Q3 = int((k+1)*0.75)
                IQR = quantile[Q3]-quantile[Q1]
                Out_low = quantile[Q1]-(IQR*1.5)
                Out_high = quantile[Q3]+(IQR*1.5)

                if(k<10):
                    ro_outcome = ro_outcome/k
                else:
                    ro_outcome = 0
                    ro_count = 0
                    for i in range(k):
                        if(neighbors[i][1]>=Out_low) and (neighbors[i][1]<=Out_high):
                            ro_outcome += neighbors[i][1]
                            ro_count += 1
                    ro_outcome = ro_outcome/ro_count
                y_predict[a]=ro_outcome

            else:
                ew_outcome = 0
                for i in range(k):
                    ew_outcome += neighbors[i][1]
                ew_outcome = ew_outcome/k
                y_predict[a]=ew_outcome
        
        ### 回傳預測結果
        return y_predict

定義 RMSE 的計算過程，提供後續使用

In [145]:
### 計算 RMSE 

def RMSE(y_pred,y_test):    
    MSE = 0
    for b in range(len(y_test)):
        MSE += (y_pred[b]-y_test[b])**2
    RMSE = (MSE/len(y_test))**0.5
    return(RMSE)

### Q1.2 

設定 k=20, mean_type = equal_weight
並輸出預測的前20筆結果，以及此次預測的 RMSE

In [147]:
### test arc 1###
myknn = myknn_regressor(20, "equal_weight")
myknn.fit(X_train, Y_train)
ypred = myknn.predict(X_test)
evaluate = RMSE(ypred,Y_test)

print(ypred[:20])
print(evaluate)

[[1990]
 [1996]
 [1997]
 [1990]
 [1989]
 [1993]
 [1990]
 [1997]
 [2001]
 [2003]
 [2000]
 [2001]
 [1995]
 [1998]
 [1988]
 [1993]
 [1990]
 [2000]
 [2002]
 [1995]]
[10.51032825]


### Q1.3 

設定 k=20, mean_type = remove_outliers
並輸出預測的前20筆結果，以及此次預測的 RMSE

In [146]:
### test arc 2###
myknn2 = myknn_regressor(20, "remove_outliers")
myknn2.fit(X_train, Y_train)
ypred2 = myknn2.predict(X_test)
evaluate2 = RMSE(ypred2,Y_test)

print(ypred2[:20])
print(evaluate2)

[[1962]
 [1975]
 [1977]
 [1989]
 [1971]
 [1970]
 [1967]
 [1996]
 [2001]
 [2003]
 [2000]
 [1992]
 [1971]
 [1992]
 [1978]
 [1978]
 [1962]
 [2000]
 [2002]
 [1995]]
[21.74039251]


### Q1 結果

在預期中，使用 remove_outliers 來計算預測結果應該要有較好的結果。
但從我寫的結果中使用 equal_weight 所算出來的 RMSE 明顯比 remove_outliers 好很多